<a href="https://colab.research.google.com/github/ericphamhung-gretel/demos/blob/main/data_designer_sdk_qa_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Data Designer SDK: Generate Diverse Q&A Pairs

This notebook demonstrates a general approach for extracting Q&A pairs from any source document
(e.g. text, markdown, or PDF files). The generated Q&A pairs can be used for:
- **Instruction Tuning:** Training models with clear, self-contained examples.
- **Retrieval-Augmented Generation (RAG):** Enhancing retrieval systems with precise and context-supported Q&A pairs.
- **Search and FAQ Systems:** Powering natural language query systems and documentation.

> **Note:** The [Data Designer](https://docs.gretel.ai/create-synthetic-data/gretel-data-designer-beta) functionality demonstrated in this notebook is currently in **Early Preview**. To access these features and run this notebook, please [join the waitlist](https://gretel.ai/navigator/data-designer#waitlist).

# 📘 Getting Started

First, let's install and import the required packages:

In [1]:
# Install required libraries

%%capture
!pip install -qq langchain unstructured[pdf] smart_open git+https://github.com/gretelai/gretel-python-client

In [2]:
# Configuration
# -------------
# Define your source document(s) and the number of Q&A pairs to generate.
# You can replace this with your own documents in PDF, markdown, or text formats.

# databricks - version, validate with alex?

DOCUMENT_LIST = ["https://www.americanexpress.com/content/dam/amex/us/rewards/membership-rewards/mr-terms-conditions-11.01.2024.pdf?mrlinknav=footer-tandc"]
NUM_QA_PAIRS = 100

In [3]:
# Document Processing
# ------------------
# The DocumentProcessor class handles loading and chunking source documents for RAG evaluation.
# We use langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from smart_open import open
import tempfile
import os

class DocumentProcessor:
    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

In [4]:
# Data Models
# -----------
# Define Pydantic models for structured output generation:
# 1. QAPair: Schema for question-answer pairs
# 2. EvalMetrics: Schema for scoring generation quality

from pydantic import BaseModel, Field
from typing import Optional, Literal

class QAPair(BaseModel):
    question: str = Field(..., description="A clear and concise question derived from the context.")
    answer: str = Field(..., description="A detailed and accurate answer fully supported by the context.")
    reasoning: str = Field(..., description="A brief explanation of why this Q&A pair is valuable.")


class EvalMetrics(BaseModel):
    clarity: int = Field(..., description="Clarity of the question", ge=1, le=5)
    factual_accuracy: int = Field(..., description="Factual accuracy of the answer", ge=1, le=5)
    comprehensiveness: int = Field(..., description="Completeness of the answer", ge=1, le=5)
    reasoning_quality: int = Field(..., description="Quality of the provided reasoning", ge=1, le=5)

In [5]:
# Setup & Configure Data Designer
# --------------------------------
# Initialize the Data Designer with a custom system message to ensure that the generated
# Q&A pairs are high-quality, context-supported, and tailored for a variety of downstream applications.


from gretel_client.navigator import DataDesigner

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Initialize Data Designer
designer = DataDesigner(
    api_key="prompt",
    model_suite="llama-3.x",  # or "apache-2.0" as needed
    endpoint="https://api.gretel.cloud",
    special_system_instructions="""\
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""
)

# Add Seed Columns for Controlled Diversity

# Context: Document chunks.
designer.add_categorical_seed_column(
    name="context",
    description="A chunk of text extracted from the source document.",
    values=chunks
)

# Difficulty with nested Sophistication.
designer.add_categorical_seed_column(
    name="focus_area",
    description="The focus area of the question.",
    values=["program overview", "earning points", "redeeming rewards", "account management", "promotion and bonuses", "terms and conditions"],
    subcategories=[
        {
            "name": "sub_focus_area",
            "values": {
                "program overview": ["introduction and description", "eligibility and enrollment", "membership tiers", "general benefits" ],
                "earning points": ["spending categories", "earning rates", "bonus offers", "promotional earning opportunities"],
                "redeeming rewards": ["redemption options", "redemption process", "points valuation and conversion rates", "restrictions and limitations"],
                "account management": ["points tracking and monitoring", "points expiration policies", "points transfer and consolidation"],
                "promotion and bonuses": ["sign-up and referral bonuses", "seasonal and limited time offers", "bonus qualification requirements"],
                "terms and conditions": ["program tules and legal terms", "fee structure and charges", "limitations and exclusions", "amendments and updates"]
            }
        }
    ]
)

# # Question Style: Tone and approach.
# designer.add_categorical_seed_column(
#     name="question_style",
#     description="The style or tone of the question.",
#     values=["factual", "exploratory", "analytical", "comparative"]
# )

# Target Audience: Language complexity.
designer.add_categorical_seed_column(
    name="target_audience",
    description="The intended audience for the Q&A pair.",
    values=["novice", "intermediate", "expert"]
)

# Response Format: Expected answer style.
# designer.add_categorical_seed_column(
#     name="response_format",
#     description="The format of the answer expected (e.g., short, detailed, step-by-step).",
#     values=["short", "detailed", "step-by-step", "list"]
# )

# Define Generation Template for Q&A Pairs
designer.add_generated_data_column(
    name="qa_pair",
    generation_prompt=(
        "\n{context}\n\n"
        "Based on the above context, generate a high-quality Q&A pair. The question should be clear and concise, "
        "and tailored for an audience at the '{target_audience}' level. The overall focus should be '{focus_area}', "
        "with a corresponding subfocus area (e.g., {sub_focus_area})."
        "Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
        "Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
        "Output the result in JSON format as follows:\n"
        '{{\n'
        '  "question": "<your question>",\n'
        '  "answer": "<your answer>",\n'
        '  "reasoning": "<your explanation>"\n'
        '}}\n'
    ),
    data_config={
        "type": "structured",
        "params": {"model": QAPair}
    }
)

# Optional: Define Evaluation Template for Q&A Pairs
designer.add_generated_data_column(
    name="eval_metrics",
    llm_type="judge",
    generation_prompt=(
        "\n{context}\n\n"
        "For the above Q&A pair:\n"
        "{qa_pair}\n\n"
        "Evaluate the following criteria on a scale of 1 (lowest) to 5 (highest):\n"
        "1. Clarity: How clear and understandable is the question?\n"
        "2. Factual Accuracy: Is the answer fully supported by the context?\n"
        "3. Comprehensiveness: Does the answer cover all necessary details?\n"
        "4. Reasoning Quality: Is the provided reasoning well-justified and clear?\n\n"
        "Output the scores in JSON format."
    ),
    data_config={
        "type": "structured",
        "params": {"model": EvalMetrics}
    }
)

# Preview a Sample of 10 Generated Records
preview = designer.generate_dataset_preview()
# preview.display_sample_record()


[17:26:51] [INFO] 🦜 Using llama-3.x model suite
Gretel API Key: ··········
Logged in as eric.phamhung@gretel.ai ✅
[17:31:11] [INFO] 🚀 Generating dataset preview
[17:31:12] [INFO] 📥 Step 1: Load data seeds
[17:31:12] [INFO] 🎲 Step 2: Sample data seeds
[17:31:13] [INFO] 🦜 Step 3: Generate column from template >> generating qa pair
[17:31:20] [INFO] 🦜 Step 4: Generate column from template >> generating eval metrics
[17:31:50] [INFO] 👀 Your dataset preview is ready for a peek!


In [6]:
from rich.console import Console
from rich.table import Table
from collections import Counter
import pandas as pd

def generate_qa_report(df):
    console = Console()
    categories = ['focus_area', 'sub_focus_area', 'target_audience', ]

    console.print("\n[bold blue]🧠 QA Pair Generation Report[/bold blue]", justify="center")
    console.print("=" * 80, justify="center")
    console.print(f"\n[bold]Total QA Pairs:[/bold] {len(df)}")

    for category in categories:
        if category in df.columns:
            # Only count non-empty values
            counts = Counter(x for x in df[category] if pd.notna(x) and x != '')
            if not counts:
                continue

            table = Table(title=f"\n{category.title()} Distribution")
            table.add_column("Category", style="cyan")
            table.add_column("Count", justify="right")
            table.add_column("Percentage", justify="right")

            total = sum(counts.values())
            for value, count in sorted(counts.items()):
                percentage = (count / total) * 100
                table.add_row(str(value), str(count), f"{percentage:.1f}%")

            console.print(table)

    if any(col.startswith('eval_metrics.') for col in df.columns):
        metrics_table = Table(title="\nQuality Metrics Summary")
        metrics_table.add_column("Metric")
        metrics_table.add_column("Average Score", justify="right")

        metrics = ['clarity', 'factual_accuracy', 'comprehensiveness', 'reasoning_quality']
        for metric in metrics:
            col = f'eval_metrics.{metric}'
            if col in df.columns:
                scores = df[col].dropna()
                if len(scores) > 0:
                    avg_score = scores.mean()
                    metrics_table.add_row(
                        metric.replace('_', ' ').title(),
                        f"{avg_score:.2f}/5.00"
                    )

        console.print(metrics_table)

generate_qa_report(preview.dataset)

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 10

                                              
           Focus_Area Distribution            
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category              ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ earning points        │     2 │      20.0% │
│ program overview      │     2 │      20.0% │
│ promotion and bonuses │     1 │      10.0% │
│ redeeming rewards     │     2 │      20.0% │
│ terms and conditions  │     3 │      30.0% │
└───────────────────────┴───────┴────────────┘

                                                         
               Sub_Focus_Area Distribution               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category                         ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ amendments and updates           │     1 │      10.0% │
│ bonus qualification requirements │     1 │      10.0% │
│ earning rates                    │     1 │      10.0% │
│ fee structure and charges        │     1 │      10.0% │
│ introduction and description     │     2 │      20.0% │
│ limitations and exclusions       │     1 │      10.0% │
│ restrictions and limitations     │     2 │      20.0% │
│ spending categories              │     1 │      10.0% │
└──────────────────────────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ expert       │     5 │      50.0% │
│ intermediate │     2 │      20.0% │
│ novice       │     3 │      30.0% │
└──────────────┴───────┴────────────┘

In [7]:
# View the preview dataset as a dataframe
preview.dataset

,context,focus_area,sub_focus_area,target_audience,qa_pair,eval_metrics
0,Use Redeem for Deposits for a deposit into you...,earning points,earning rates,novice,"{""question"": ""What is the earning rate for red...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
1,11\n\nLimitation on Arbitration\n\nArbitration...,promotion and bonuses,bonus qualification requirements,expert,"{""question"": ""What is the minimum award amount...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,The points you got for a purchase are reversed...,terms and conditions,fee structure and charges,intermediate,"{""question"": ""How do I submit a claim to media...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,The points you got for a purchase are reversed...,redeeming rewards,restrictions and limitations,novice,"{""question"": ""What happens if I don't have eno...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
4,1 additional point (for a total of 2 points) o...,terms and conditions,amendments and updates,expert,"{""question"": ""What types of purchases are excl...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
5,1\n\nCorporate Cards issued to more than one i...,program overview,introduction and description,expert,"{""question"": ""What are the key aspects of the ...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
6,1 additional point (for a total of 2 points) o...,earning points,spending categories,expert,"{""question"": ""What are the spending categories...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
7,Use Redeem for Deposits for a deposit into you...,redeeming rewards,restrictions and limitations,intermediate,"{""question"": ""How do restrictions and limitati...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
8,Use Redeem for Deposits for a deposit into you...,program overview,introduction and description,novice,"{""question"": ""What is the Redeem for Deposits ...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
9,The points you got for a purchase are reversed...,terms and conditions,limitations and exclusions,expert,"{""question"": ""What are the limitations and exc...","{""clarity"": 5, ""factual_accuracy"": 2, ""compreh..."


In [8]:
preview.display_sample_record(1)

                                             Categorical Seed Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ context             │ 11                                                                                        │
│                     │                                                                                           │
│                     │ Limitation on Arbitration                                                                 │
│                     │                                                                                           │
│                     │ Arbitration Fees and Costs                                                                │
│                     │                                                                                           │
│                     │ If the parties agree to resolve a claim by arbitration, that claim will be arbitrated on  │
│                     │ an individual basis pursuant to that agreement, and the agreement would not allow claims  │
│                     │ to be arbitrated on a class action basis or on bases involving claims brought in a        │
│                     │ purported representative capacity on behalf of the general public, other Card Members or  │
│                     │ other persons similarly situated. The arbitrator's authority is limited to claims between │
│                     │ you and us alone. Claims may not be joined or consolidated unless you and we agree in     │
│                     │ writing. An arbitration award and any judgment confirming it will apply only to the       │
│                     │ specific case and cannot be used in any other case except to enforce the award.           │
│                     │ Notwithstanding any other provision and without waiving the right to appeal such          │
│                     │ decision, if any portion of these Limitations on Arbitration provisions is deemed invalid │
│                     │ or unenforceable, then the entire Arbitration provision (other than this sentence) will   │
│                     │ not apply.                                                                                │
│                     │                                                                                           │
│                     │ You will be responsible for paying your share of any arbitration fees (including filing,  │
│                     │ administrative, hearing or other fees), but only up to the amount of the filing fees you  │
│                     │ would have incurred if you had brought a claim in court. We will be responsible for any   │
│                     │ additional arbitration fees.                                                              │
│                     │                                                                                           │
│                     │ At your written request, we will consider in good faith making a temporary advance of     │
│                     │ your share of any arbitration fees, or paying for the reasonable fees of an expert        │
│                     │ appointed by the arbitrator for good cause.                                               │
│                     │                                                                                           │
│                     │ Additional Arbitration Awards                                                             │
│                     │                                                                                           │
│                     │ If the arbitrator rules in your 

In [9]:
# # Generate and Analyze Dataset
# # ---------------------------
# # Uncomment these lines to generate evaluation pairs, save them to JSONL,
# # and analyze the coverage and quality of the generated dataset.

batch_job = designer.submit_batch_workflow(num_records=NUM_QA_PAIRS)
dataset = batch_job.fetch_dataset(wait_for_completion=True)

generate_qa_report(dataset)

dataset.to_json('qa_pairs.jsonl', orient='records', lines=True)


[17:31:50] [INFO] ⚙️ Configuring Data Designer Workflow steps:
[17:31:50] [INFO]   |-- Step 1: load-data-seeds-1
[17:31:50] [INFO]   |-- Step 2: sample-data-seeds-2
[17:31:50] [INFO]   |-- Step 3: generate-column-from-template-3-generating-qa-pair
[17:31:50] [INFO]   |-- Step 4: generate-column-from-template-4-generating-eval-metrics
[17:31:52] [INFO] 🛜 Connecting to your Gretel Project:
[17:31:52] [INFO] 🔗 -> https://console.gretel.ai/proj_2swzdW9XU5Xuldo2716xQBLsSae
[17:31:53] [INFO] ▶️ Starting your workflow run to generate 100 records:
[17:31:53] [INFO]   |-- project_name: gretel-sdk-73e48-44ab09eab0fa0bb
[17:31:53] [INFO]   |-- project_id: proj_2swzdW9XU5Xuldo2716xQBLsSae
[17:31:53] [INFO]   |-- workflow_run_id: wr_2swzdhqkUlPlxReOzHb5NLwwfU6
[17:31:53] [INFO] 🔗 -> https://console.gretel.ai/workflows/w_2swzdi0mha6X36fp4CZXvHtAmhc/runs/wr_2swzdhqkUlPlxReOzHb5NLwwfU6
[17:31:56] [INFO] ⏳ Waiting for workflow step `generate-column-from-template-4-generating-eval-metrics` to complete..

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 100

                                              
           Focus_Area Distribution            
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category              ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ account management    │    13 │      13.0% │
│ earning points        │    26 │      26.0% │
│ program overview      │    19 │      19.0% │
│ promotion and bonuses │    10 │      10.0% │
│ redeeming rewards     │    14 │      14.0% │
│ terms and conditions  │    18 │      18.0% │
└───────────────────────┴───────┴────────────┘

                                                              
                 Sub_Focus_Area Distribution                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category                              ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ amendments and updates                │     6 │       6.0% │
│ bonus offers                          │     7 │       7.0% │
│ bonus qualification requirements      │     5 │       5.0% │
│ earning rates                         │     6 │       6.0% │
│ eligibility and enrollment            │     6 │       6.0% │
│ fee structure and charges             │     6 │       6.0% │
│ general benefits                      │     4 │       4.0% │
│ introduction and description          │     5 │       5.0% │
│ limitations and exclusions            │     2 │       2.0% │
│ membership tiers                      │     4 │       4.0% │
│ points expiration policies            │     6 │       6.0% │
│ points tracking and monitoring        │     4 │       4.0% │
│ points transfer and consolidation     │     3 │       3.0% │
│ points valuation and conversion rates │     6 │       6.0% │
│ program tules and legal terms         │     4 │       4.0% │
│ promotional earning opportunities     │     4 │       4.0% │
│ redemption options                    │     3 │       3.0% │
│ redemption process                    │     3 │       3.0% │
│ restrictions and limitations          │     2 │       2.0% │
│ seasonal and limited time offers      │     3 │       3.0% │
│ sign-up and referral bonuses          │     2 │       2.0% │
│ spending categories                   │     9 │       9.0% │
└───────────────────────────────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ expert       │    39 │      39.0% │
│ intermediate │    29 │      29.0% │
│ novice       │    32 │      32.0% │
└──────────────┴───────┴────────────┘

In [10]:
dataset

,context,focus_area,sub_focus_area,target_audience,qa_pair,eval_metrics
0,11\n\nLimitation on Arbitration\n\nArbitration...,earning points,promotional earning opportunities,novice,"{""question"": ""If I choose to arbitrate a claim...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
1,3. Gasoline at gas stations in the U.S. (not i...,account management,points transfer and consolidation,intermediate,"{""question"": ""Can you change your choice of ca...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
2,1 additional point (for a total of 2 points) o...,program overview,general benefits,novice,"{""question"": ""What are the general benefits of...","{""clarity"": 5, ""comprehensiveness"": 4, ""factua..."
3,3. Gasoline at gas stations in the U.S. (not i...,redeeming rewards,points valuation and conversion rates,novice,"{""question"": ""How can I earn additional points...","{""clarity"": 5, ""comprehensiveness"": 4, ""factua..."
4,Welcome to Membership Rewards!\n\nLast Updated...,earning points,bonus offers,expert,"{""question"": ""What happens to linked Products ...","{""clarity"": 4, ""comprehensiveness"": 3, ""factua..."
...,...,...,...,...,...,...
95,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,program overview,general benefits,novice,"{""question"": ""What are the general benefits of...","{""clarity"": 4, ""factual_accuracy"": 4, ""compreh..."
96,Use Redeem for Deposits for a deposit into you...,terms and conditions,fee structure and charges,expert,"{""question"": ""What is the excise tax offset fe...","{""clarity"": 5, ""comprehensiveness"": 4, ""factua..."
97,"Also, some products will earn additional point...",redeeming rewards,redemption options,expert,"{""question"": ""What specific conditions must be...","{""clarity"": 4, ""comprehensiveness"": 4, ""factua..."
98,If you don't make a timely payment of the mini...,account management,points tracking and monitoring,intermediate,"{""question"": ""How can I recover points that I ...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."


In [11]:
#preview
import json
parsed = json.loads(dataset['qa_pair'][99])
print(json.dumps(parsed, indent=4))

{
    "question": "What happens if any portion of the Claims Resolution section is deemed invalid or unenforceable, and how does this affect the remaining portions of the section?",
    "answer": "If any portion of the Claims Resolution section, except as otherwise provided in the Limitations on Arbitration subsection, is deemed invalid or unenforceable, it will not invalidate the remaining portions of this Claims Resolution section.",
    "reasoning": "This Q&A pair is valuable as it highlights the importance of understanding the contractual obligations and the implications of invalidating any portion of the Claims Resolution section. It demonstrates the arbitrator's intention to preserve the integrity of the section, ensuring that the remaining provisions remain enforceable."
}


# Using OpenAI

In [12]:
from openai import OpenAI
from google.colab import userdata


In [13]:
client = OpenAI(
    base_url="https://litellm-proxy.dev.gretel.cloud/v1",
    api_key=userdata.get("GRETEL_API_KEY")
)

# [m.id for m in client.models.list()]

In [14]:
system_prompt = """
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""

user_prompt = f'''
"\n{chunks}\n\n"
"Based on the above context, generate {NUM_QA_PAIRS} high-quality Q&A pair. The question should be clear and concise, "
"and tailored for an audience at either the ["novice", "intermediate", "expert"] level.
The overall focus should be either ["program overview", "earning points", "redeeming rewards", "account management", "promotion and bonuses", "terms and conditions"], "
"with a corresponding subfocus area "
"Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
"Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
"Output the result in JSON format as follows:\n"
'{{\n'
'  "question": "<your question>",\n'
'  "answer": "<your answer>",\n'
'  "target_audience": "<target audience>",\n'
'  "focus_area": "<focus area>",\n'
'  "sub_focus_area": "<subfocus area>",\n'
'  "reasoning": "<your explanation>"\n'
'}}\n'


'''

In [15]:
completion = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)
# print(completion.choices[0].message)




In [16]:
print(completion.choices[0].message.content)

{
  "qa_pairs": [
    {
      "question": "What entity owns and operates the Membership Rewards Program?",
      "answer": "The Membership Rewards Program is owned and operated by American Express Travel Related Services Company, Inc.",
      "target_audience": "novice",
      "focus_area": "program overview",
      "sub_focus_area": "general information",
      "reasoning": "This question provides foundational information about the entity responsible for the Membership Rewards Program, essential for new members."
    },
    {
      "question": "Can American Express change the number of points required to redeem rewards?",
      "answer": "Yes, American Express may change the number of points required to redeem for rewards at any time.",
      "target_audience": "intermediate",
      "focus_area": "redeeming rewards",
      "sub_focus_area": "point values",
      "reasoning": "This addresses a key concern for members regarding potential changes in reward redemption values."
    },
    

In [ ]:
# prompt: parse json into dataframe

import pandas as pd
import json

# Assuming 'dataset' is your DataFrame from the previous code
# and it contains a 'qa_pair' column with JSON strings.

def parse_json_to_dataframe(dataset):
    """Parses JSON strings in the 'qa_pair' column into a new DataFrame.

    Args:
        dataset: The input DataFrame with a 'qa_pair' column containing JSON strings.

    Returns:
        A new DataFrame with the parsed JSON data or None if parsing fails.
    """

    try:
        parsed_data = []
        for index, row in dataset.iterrows():
            try:
                qa_json_string = row['qa_pair']
                parsed_json = json.loads(qa_json_string)
                parsed_data.append(parsed_json)
            except json.JSONDecodeError as e:
                print(f"Skipping row {index} due to JSONDecodeError: {e}")
                continue  # Skip to the next row if parsing fails
        parsed_df = pd.DataFrame(parsed_data)
        return parsed_df
    except (KeyError, TypeError):
        print("'qa_pair' column not found or invalid data type.")
        return None

# Example usage:
# Assuming 'dataset' DataFrame is already defined from previous code
parsed_df = parse_json_to_dataframe(dataset)

if parsed_df is not None:
    print(parsed_df.head())


In [27]:
parsed_json = json.loads(completion.choices[0].message.content)['qa_pairs']


In [31]:
# prompt: turn a list of dicts into a dataframe, where each key is a column. the list is parsed_json

import pandas as pd
import json

def parse_json_to_dataframe(json_list):
    """Parses a list of JSON objects into a Pandas DataFrame.

    Args:
        json_list: A list of dictionaries representing JSON objects.

    Returns:
        A Pandas DataFrame, or None if an error occurs.
    """
    try:
        return pd.DataFrame(json_list)
    except (ValueError, TypeError) as e:
        print(f"Error creating DataFrame: {e}")
        return None

# Example usage (assuming parsed_json is defined as in your provided code):
# parsed_json = json.loads(completion.choices[0].message.content)['qa_pairs']
df = parse_json_to_dataframe(parsed_json)


In [34]:
generate_qa_report(df)

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 20

                                             
           Focus_Area Distribution           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category             ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ account management   │     2 │      10.0% │
│ earning points       │     3 │      15.0% │
│ program overview     │     1 │       5.0% │
│ redeeming rewards    │    12 │      60.0% │
│ terms and conditions │     2 │      10.0% │
└──────────────────────┴───────┴────────────┘

                                                     
             Sub_Focus_Area Distribution             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category                     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ Amazon integration           │     1 │       5.0% │
│ Pay with Points              │     1 │       5.0% │
│ available options            │     1 │       5.0% │
│ debit card usage             │     1 │       5.0% │
│ eligibility                  │     1 │       5.0% │
│ fraud and mismanagement      │     1 │       5.0% │
│ general information          │     1 │       5.0% │
│ gift cards and certificates  │     1 │       5.0% │
│ limitations                  │     1 │       5.0% │
│ merchant categories          │     1 │       5.0% │
│ partner loyalty programs     │     1 │       5.0% │
│ payment issues               │     1 │       5.0% │
│ point transfers              │     1 │       5.0% │
│ point values                 │     1 │       5.0% │
│ product linking              │     1 │       5.0% │
│ product-specific limitations │     1 │       5.0% │
│ program changes              │     1 │       5.0% │
│ timing                       │     1 │       5.0% │
│ transfers and limitations    │     1 │       5.0% │
│ value determination          │     1 │       5.0% │
└──────────────────────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ advanced     │     1 │       5.0% │
│ expert       │     4 │      20.0% │
│ intermediate │    10 │      50.0% │
│ novice       │     5 │      25.0% │
└──────────────┴───────┴────────────┘

# Evaluation

1. finetune and test on a holdout dataset?
  - BLEU/ROUGE scores? While BLEU score is primarily used for machine translation tasks, ROUGE score is used for text summarization tasks
2. comparing intrinsic properties of the datasets:
  - diversity coverage (topic distribution, keyword distributions, question types, answer styles, content analysis)
  - quality annotations (ambiguitiy detection, error analysis)
  - linguistic and structural properties (readability scores, synctactic complexity, length and structure distribution)
  - vocabularity diversity (type/token ratio, vocabularity distribution)
  - semantic consistancy
  - redudancy and duplication checks


value add?
-